In [ ]:
import pandas as pd
import numpy as np
import math
import datetime
import subprocess
import bandit.main as bd

In [ ]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [ ]:
# todo: combine bandit_class and bandit_diff
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [ ]:
def df_bandit_diff(df):
    dfgb = df.groupby(['goal'])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [ ]:
subprocess.Popen('GoalSetter.xlsx', shell=True)

In [ ]:
df = get_frame('GoalSetter.xlsx')

In [ ]:
df = bd.reduce(df,'goal')

In [ ]:
df_quality = bd.get_quality(df['score'])

In [ ]:
df = df.join(df_quality).sort_values('Quality', ascending=False)

In [ ]:
#df.loc[df.score.isnull(), 'Quality'] = 0.5

In [ ]:
df.sort_values('date', ascending=True).tail(10)

In [ ]:
df = df.sort_values('Quality', ascending=False)
df.head(10)

In [ ]:
df_class = df_bandit_class(df,'cohort')

In [ ]:
df_class

In [ ]:
nextClass = df_class.iloc[0,0]

In [ ]:
if df['complete'].isnull().sum() > 0:
    df_filtered = df[df['cohort'] == nextClass]
else:
    df_filtered = df

In [ ]:
df_bandit_diff(df_filtered)